In [83]:
from pptx import Presentation
from pptx.util import Pt, Inches
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
import pandas as pd
from datetime import datetime, timedelta
from functools import reduce
import os
from pptx.chart.data import CategoryChartData
from pptx.enum.chart import XL_CHART_TYPE
from pptx.enum.chart import XL_MARKER_STYLE, XL_LABEL_POSITION


In [84]:
######                              DATE DATA                   #####

start_date = datetime(2024, 9, 29)
end_date = start_date + timedelta(days=6)
analysis_date = '11th Oct\'24'

In [45]:
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return RGBColor(int(hex_color[0:2], 16), int(hex_color[2:4], 16), int(hex_color[4:6], 16))

In [46]:
def merge_dfs(df_list, on_column):
    return reduce(lambda left, right: pd.merge(left, right, on=on_column, how='outer'), df_list)

In [47]:
def text_formatter(text_frame, font_size, font_name, alignment, font_color):
    for paragraph in text_frame.paragraphs:
        for run in paragraph.runs:
            run.font.size = Pt(font_size)
            run.font.name = font_name
            if isinstance(font_color, str):
                run.font.color.rgb = RGBColor.from_string(font_color.replace('#', ''))
            elif isinstance(font_color, RGBColor):
                run.font.color.rgb = font_color
        paragraph.alignment = alignment

In [48]:
def add_date_textbox(ppt, slide_number, left, top, width, height, text, font_size, font_color_hex, font):

    font_color = hex_to_rgb(font_color_hex)
    textbox = ppt.slides[slide_number].shapes.add_textbox(left, top, width, height)
    
    p = textbox.text_frame.add_paragraph()
    p.text = text
    p.font.size = Pt(font_size)
    p.font.color.rgb = font_color
    p.font.name = font

In [49]:
def mil_k_mapper(value):
    if value / 1000000 > 1:
        return str(round(value / 1000000, 1)) + 'M'
    else:
        return str(round(value / 1000, 1)) + 'k'

In [50]:
mapping_df = pd.read_excel(r"F:\Projects\PPTX_env\WB_weekly\Party_mapping.xlsx")
last_7_days_path = r"C:\Users\useer\Downloads\WB_Sep_29_Oct_5_data_dump\WB_Report_data_2024-10-06.xlsx"
overall_path = r"C:\Users\useer\Downloads\WB_Sep_29_Oct_5_data_dump\WB_Report_data_2024-10-06_overall.xlsx"

In [85]:
# Read Template
ppt = Presentation(r"F:\Projects\PPTX_env\WB_weekly\WB_Weekly SMA_template.pptx")

In [52]:
# Slide 1
text_box_count = 0
for shape in ppt.slides[0].shapes:
    if shape.has_text_frame:
        text_box_count += 1

    if text_box_count == 3:
        shape.text = 'Analysis Date:' + analysis_date
        shape.text_frame.paragraphs[0].runs[0].font.name = 'Bahnschrift'
        shape.text_frame.paragraphs[0].runs[0].font.size = Pt(20)

    if shape.has_table:
        shape.table.rows[0].cells[1].text = start_date.strftime("%d %b, %Y")
        shape.table.rows[0].cells[3].text = end_date.strftime("%d %b, %Y")

In [53]:
platform_metric_sheets = ['Facebook_all_stats_post', 'Instagram_all_stats_post', 'Twitter_all_stats_post', 'YouTube_all_stats_post']

dfs_last_7 = [pd.read_excel(last_7_days_path, sheet_name=sheet) for sheet in platform_metric_sheets]
dfs_overall = [pd.read_excel(overall_path, sheet_name=sheet) for sheet in platform_metric_sheets]

In [54]:
for i, sheet_name in enumerate(platform_metric_sheets):
    dfs_last_7[i] = dfs_last_7[i][dfs_last_7[i]['mapping'] == 'Party']

    if i == 3:
        dfs_last_7[i] = dfs_last_7[i][['Name', 'Followers', 'Avg Video Views', 'Avg Engagement', 'Engagement Rate']]
    else:
        dfs_last_7[i] = dfs_last_7[i][['Name', 'Followers', 'Avg. Post Per Day', 'Avg Engagement', 'Engagement Rate']]

    dfs_last_7[i] = dfs_last_7[i].dropna(subset = 'Followers')
    dfs_last_7[i]['Name'] = pd.Categorical(dfs_last_7[i]['Name'], categories = mapping_df['Name'].tolist(), ordered=True)
    dfs_last_7[i] = dfs_last_7[i].sort_values('Name')
    dfs_last_7[i].columns = [dfs_last_7[i].columns[0]] + [f"{col}_{sheet_name[:-15]}" for col in dfs_last_7[i].columns[1:]]

last_7_df_party = merge_dfs(dfs_last_7, 'Name')

In [55]:
for i, sheet_name in enumerate(platform_metric_sheets):
    dfs_overall[i] = dfs_overall[i][dfs_overall[i]['mapping'] == 'Party']
    # dfs_overall[i] = pd.merge(dfs_overall[i], mapping_df, how='left', left_on = 'presence_handle', right_on = sheet_name[:-15])

    if i == 3:
        dfs_overall[i] = dfs_overall[i][['Name', 'Followers', 'Avg Video Views', 'Avg Engagement', 'Engagement Rate']]
        # dfs_overall[i] = dfs_overall[i].rename(columns={'presence_handle': 'Name'})
    else:
        dfs_overall[i] = dfs_overall[i][['Name', 'Followers', 'Avg. Post Per Day', 'Avg Engagement', 'Engagement Rate']]

    dfs_overall[i] = dfs_overall[i].dropna(subset = 'Followers')
    dfs_overall[i]['Name'] = pd.Categorical(dfs_overall[i]['Name'], categories = mapping_df['Name'].tolist(), ordered=True)
    dfs_overall[i] = dfs_overall[i].sort_values('Name')
    dfs_overall[i].columns = [dfs_overall[i].columns[0]] + [f"{col}_{sheet_name[:-15]}" for col in dfs_overall[i].columns[1:]]

overall_df_party = merge_dfs(dfs_overall, 'Name')

In [56]:
# Slide 7
table_count = 0

for shape in ppt.slides[9].shapes:
    if shape.has_table:
        table_count += 1

        if table_count == 2:
            for row_index, row in enumerate(shape.table.rows):
                if row_index > 2:
                    for col_index, cell in enumerate(row.cells):
                        if col_index > 0:
                            if pd.isna(overall_df_party.iloc[row_index - 3, col_index]):
                                cell.text = '-'
                                text_formatter(cell.text_frame, 11, "Bahnschrift", PP_ALIGN.CENTER, "#000000")
                            else:
                                if col_index in [1, 5, 9, 13]:
                                    cell.text = f"{mil_k_mapper(overall_df_party.iloc[row_index - 3, col_index])}"
                                else:
                                    cell.text = f"{overall_df_party.iloc[row_index - 3, col_index]:,.1f}"
                                text_formatter(cell.text_frame, 11, "Bahnschrift", PP_ALIGN.CENTER, "#000000")

        if table_count == 1:
            for row_index, row in enumerate(shape.table.rows):
                if row_index > 2:
                    for col_index, cell in enumerate(row.cells):
                        if col_index > 0:
                            if pd.isna(last_7_df_party.iloc[row_index - 3, col_index]):
                                cell.text = '-'
                                text_formatter(cell.text_frame, 11, "Bahnschrift", PP_ALIGN.CENTER, "#000000")
                            else:
                                if col_index in [1, 5, 9, 13]:
                                    cell.text = f"{mil_k_mapper(last_7_df_party.iloc[row_index - 3, col_index])}"
                                else:
                                    cell.text = f"{last_7_df_party.iloc[row_index - 3, col_index]:,.1f}"
                                text_formatter(cell.text_frame, 11, "Bahnschrift", PP_ALIGN.CENTER, "#000000")

In [86]:
ppt_image_order_party = ['Followers', 'count_of_active_days', 'Count of Active Days', 'Total Post', 'Total Posts', 'Avg. Post Per Day', 'Total Engagement', 'Avg Engagement', 'Engagement Rate', 'Total Video Views', 'Avg Video Views', 'impressions']
platform_order = ['Facebook', 'Twitter', 'Instagram', 'YouTube']

Last 7 days

In [58]:
# Slide 17 - 27
party_metric_slide_index = 16
platform_index = 0
delta_x = Inches(2.28)
x0 = Inches(1.08) 
y0 = Inches(2.07)
w = Inches(2.39)  
h = Inches(1.24)
starter_bias = Inches(0.47)

while party_metric_slide_index < 27:
    marker_count = 0
    is_starter_slide = True
    for index, image in enumerate(ppt_image_order_party):
        image_path = rf"F:\Projects\PPTX_env\WB_weekly\bar_chart_images\{platform_order[platform_index]}\Party\{image}.png"
        try:
            if image in ['Followers', 'Total Engagement']:
                ppt.slides[party_metric_slide_index].shapes.add_picture(image_path, x0 + marker_count * delta_x - starter_bias, y0, width = w * 1.23, height=h)
            else:
                ppt.slides[party_metric_slide_index].shapes.add_picture(image_path, x0 + marker_count * delta_x, y0, width=w, height=h)
            marker_count += 1
        except:
            pass

        if marker_count == 4 and is_starter_slide:
            party_metric_slide_index += 1
            marker_count = 0
            is_starter_slide = False
            
    if platform_index != 3:
        ppt.slides[party_metric_slide_index - 1].shapes.add_picture(rf"F:\Projects\PPTX_env\WB_weekly\bar_chart_images\{platform_order[platform_index]}\Party\Post_distribution.png", x0 + 4 * delta_x, y0, width = w, height = h)
        
    platform_index += 1
    marker_count = 0
    party_metric_slide_index += 2

Changes

In [59]:
# Slide 17 - 27
party_metric_slide_index = 16
platform_index = 0
delta_x = Inches(2.28)
x0 = Inches(1.08) 
y0 = Inches(4.28)
w = Inches(2.39)  
h = Inches(1.24)
starter_bias = Inches(0.47)

while party_metric_slide_index < 27:
    marker_count = 0
    is_starter_slide = True
    for index, image in enumerate(ppt_image_order_party):
        image_path = rf"F:\Projects\PPTX_env\WB_weekly\bar_chart_images\{platform_order[platform_index]}_delta_metrics\Party\{image}.png"
        try:
            if image in ['Followers', 'Total Engagement']:
                ppt.slides[party_metric_slide_index].shapes.add_picture(image_path, x0 + marker_count * delta_x - starter_bias, y0, width = w * 1.23, height=h)
            else:
                ppt.slides[party_metric_slide_index].shapes.add_picture(image_path, x0 + marker_count * delta_x, y0, width=w, height=h)
            marker_count += 1
        except:
            pass

        if marker_count == 4 and is_starter_slide:
            party_metric_slide_index += 1
            marker_count = 0
            is_starter_slide = False

    platform_index += 1
    marker_count = 0
    party_metric_slide_index += 2

In [60]:
platform_metric_sheets = ['Facebook_all_stats_post', 'Instagram_all_stats_post', 'Twitter_all_stats_post', 'YouTube_all_stats_post']

dfs_last_7 = [pd.read_excel(last_7_days_path, sheet_name=sheet) for sheet in platform_metric_sheets]
dfs_overall = [pd.read_excel(overall_path, sheet_name=sheet) for sheet in platform_metric_sheets]

In [61]:
for i, sheet_name in enumerate(platform_metric_sheets):
    dfs_last_7[i] = dfs_last_7[i][dfs_last_7[i]['mapping'] == 'Leader']
    # dfs_last_7[i] = pd.merge(dfs_last_7[i], mapping_df, how='left', left_on = 'presence_handle', right_on = sheet_name[:-15])

    if i == 3:
        dfs_last_7[i] = dfs_last_7[i][['Name', 'Followers', 'Avg Video Views', 'Avg Engagement', 'Engagement Rate']]
        # dfs_last_7[i] = dfs_last_7[i].rename(columns={'presence_handle': 'Name'})
    else:
        dfs_last_7[i] = dfs_last_7[i][['Name', 'Followers', 'Avg. Post Per Day', 'Avg Engagement', 'Engagement Rate']]
        
    dfs_last_7[i] = dfs_last_7[i].dropna(subset = 'Followers')
    dfs_last_7[i]['Name'] = pd.Categorical(dfs_last_7[i]['Name'], categories = mapping_df['Name'].tolist(), ordered=True)
    dfs_last_7[i] = dfs_last_7[i].sort_values('Name')
    dfs_last_7[i].columns = [dfs_last_7[i].columns[0]] + [f"{col}_{sheet_name[:-15]}" for col in dfs_last_7[i].columns[1:]]

last_7_df_leader = merge_dfs(dfs_last_7, 'Name')

In [62]:
for i, sheet_name in enumerate(platform_metric_sheets):
    dfs_overall[i] = dfs_overall[i][dfs_overall[i]['mapping'] == 'Leader']
    # dfs_overall[i] = pd.merge(dfs_overall[i], mapping_df, how='left', left_on = 'presence_handle', right_on = sheet_name[:-15])

    if i == 3:
        dfs_overall[i] = dfs_overall[i][['Name', 'Followers', 'Avg Video Views', 'Avg Engagement', 'Engagement Rate']]
        # dfs_overall[i] = dfs_overall[i].rename(columns={'presence_handle': 'Name'})
    else:
        dfs_overall[i] = dfs_overall[i][['Name', 'Followers', 'Avg. Post Per Day', 'Avg Engagement', 'Engagement Rate']]

    dfs_overall[i] = dfs_overall[i].dropna(subset = 'Followers')
    dfs_overall[i]['Name'] = pd.Categorical(dfs_overall[i]['Name'], categories = mapping_df['Name'].tolist(), ordered=True)
    dfs_overall[i] = dfs_overall[i].sort_values('Name')
    dfs_overall[i].columns = [dfs_overall[i].columns[0]] + [f"{col}_{sheet_name[:-15]}" for col in dfs_overall[i].columns[1:]]

overall_df_leader = merge_dfs(dfs_overall, 'Name')

In [87]:
target_leader_list = ['Mamata Banerjee', 'Abhishek Banerjee', 'Suvendu Adhikari', 'Dilip Ghosh', 'Adhir Ranjan Chowdhury', 'Md Salim', 'Sujan Chakraborty']

In [64]:
last_7_df_leader = last_7_df_leader[last_7_df_leader['Name'].isin(target_leader_list)]
overall_df_leader = overall_df_leader[overall_df_leader['Name'].isin(target_leader_list)]

In [65]:
# Slide 35
text_box_count = 0

for shape in ppt.slides[35].shapes:

    if shape.has_table:

        for row_index, row in enumerate(shape.table.rows):
            if row_index > 2:
                for col_index, cell in enumerate(row.cells):
                    if col_index > 1:
                        if pd.isna(last_7_df_leader.iloc[row_index - 3, col_index - 1]):
                            cell.text = '-'
                            text_formatter(cell.text_frame, 11, "Bahnschrift", PP_ALIGN.CENTER, "#000000")
                        else:
                            if col_index in [2, 6, 10, 14]:
                                cell.text = f"{mil_k_mapper(last_7_df_leader.iloc[row_index - 3, col_index - 1])}"
                            else:
                                cell.text = f"{last_7_df_leader.iloc[row_index - 3, col_index - 1]:,.1f}"
                            text_formatter(cell.text_frame, 11, "Bahnschrift", PP_ALIGN.CENTER, "#000000")

In [66]:
# Slide 36
text_box_count = 0

for shape in ppt.slides[34].shapes:

    if shape.has_table:

        for row_index, row in enumerate(shape.table.rows):
            if row_index > 2:
                for col_index, cell in enumerate(row.cells):
                    if col_index > 1:
                        if pd.isna(overall_df_leader.iloc[row_index - 3, col_index - 1]):
                            cell.text = '-'
                            text_formatter(cell.text_frame, 11, "Bahnschrift", PP_ALIGN.CENTER, "#000000")
                        else:
                            if col_index in [2, 6, 10, 14]:
                                cell.text = f"{mil_k_mapper(overall_df_leader.iloc[row_index - 3, col_index - 1])}"
                            else:
                                cell.text = f"{overall_df_leader.iloc[row_index - 3, col_index - 1]:,.1f}"
                            text_formatter(cell.text_frame, 11, "Bahnschrift", PP_ALIGN.CENTER, "#000000")

In [67]:
ppt_image_order_leader = ['Followers', 'count_of_active_days', 'Count of Active Days', 'Total Post', 'Total Posts', 'Avg. Post Per Day', 'Total Video Views', 'Avg. Views', 'Post_distribution', 'Avg Video Views', 'Total Engagement', 'Avg Engagement', 'Engagement Rate', 'impressions']

Normal vertical bar charts (Leader)

In [68]:
# Slide 47 - 62
leader_metric_slide_index = 47
platform_index = 0
delta_x = Inches(3.83)    # x0 + delta_x moves 2 charts places
x0 = Inches(1.65) 
y0 = Inches(1.52)
w = Inches(1.99)  
h = Inches(5.34)
starter_bias = Inches(1.5)

while leader_metric_slide_index < 63:
    if platform_index == 3:
        h = h * 0.75
    marker_count = 0
    for index, image in enumerate(ppt_image_order_leader):
        image_path = rf"F:\Projects\PPTX_env\WB_weekly\bar_chart_images\{platform_order[platform_index]}\Leader\{image}.png"
        try:
            if image in ['Followers', 'Total Engagement', 'Avg. Post Per Day', 'impressions']:
                ppt.slides[leader_metric_slide_index].shapes.add_picture(image_path, x0 + marker_count * delta_x - starter_bias, y0, width = w * 1.68, height=h)  
            elif image == 'Avg Video Views' and platform_order[platform_index] != 'YouTube':
                ppt.slides[leader_metric_slide_index].shapes.add_picture(image_path, x0 + (marker_count - 0.5) * delta_x, y0, width=w, height=h)    
            else:
                ppt.slides[leader_metric_slide_index].shapes.add_picture(image_path, x0 + marker_count * delta_x, y0, width=w, height=h)

            marker_count += 1
        except:
            if platform_order[platform_index] == 'Instagram' and image == 'Avg Video Views':
                marker_count += 1
            pass

        if marker_count % 3 == 0:
            if marker_count == 0 and platform_order[platform_index] == 'YouTube':
                pass
            else:
                leader_metric_slide_index += 1
                marker_count = 0

    platform_index += 1
    
    if marker_count != 0:
        leader_metric_slide_index += 2

% Change metrics for vertical bar charts (Leader)

In [69]:
# Slide 47 - 62
leader_metric_slide_index = 47
platform_index = 0
delta_x = Inches(3.83)
x0 = Inches(3.57) 
y0 = Inches(1.52)
w = Inches(1.99)  
h = Inches(5.34)

while leader_metric_slide_index < 63:
    if platform_index == 3:
        h = h * 0.75
    marker_count = 0
    for index, image in enumerate(ppt_image_order_leader):
        if image == 'Post_distribution' and platform_order[platform_index] != 'YouTube':
            marker_count += 1
        image_path = rf"F:\Projects\PPTX_env\WB_weekly\bar_chart_images\{platform_order[platform_index]}_delta_metrics\Leader\{image}.png"
        try:
            if image == 'Avg Video Views' and platform_order[platform_index] != 'YouTube':
                ppt.slides[leader_metric_slide_index].shapes.add_picture(image_path, x0 + (marker_count - 0.5) * delta_x, y0, width=w, height=h)
            else:    
                ppt.slides[leader_metric_slide_index].shapes.add_picture(image_path, x0 + marker_count * delta_x, y0, width=w, height=h)
            marker_count += 1
        except:
            if platform_order[platform_index] == 'Instagram' and image == 'Avg Video Views':
                marker_count += 1
            pass

        if marker_count % 3 == 0:
            if marker_count == 0 and platform_order[platform_index] == 'YouTube':
                pass
            else:
                leader_metric_slide_index += 1
                marker_count = 0

    platform_index += 1
    
    if marker_count != 0:
        leader_metric_slide_index += 2

Normal vertical bar charts (Campaign)

In [70]:
# Slide 66 - 80
leader_metric_slide_index = 65
platform_index = 0
delta_x = Inches(3.83)
x0 = Inches(1.65) 
y0 = Inches(1.52)
w = Inches(1.99)  
h = Inches(1.24)

while leader_metric_slide_index < 80:
    marker_count = 0
    for index, image in enumerate(ppt_image_order_leader):
        image_path = rf"F:\Projects\PPTX_env\WB_weekly\bar_chart_images\{platform_order[platform_index]}\Campaign\{image}.png"
        try:
            if image in ['Followers', 'Total Engagement', 'Avg. Post Per Day', 'impressions']:
                ppt.slides[leader_metric_slide_index].shapes.add_picture(image_path, x0 + marker_count * delta_x - starter_bias, y0, width = w * 1.68, height=h)
            elif image == 'Avg Video Views' and platform_order[platform_index] != 'YouTube':
                ppt.slides[leader_metric_slide_index].shapes.add_picture(image_path, x0 + (marker_count - 0.5) * delta_x, y0, width=w, height=h)
            else:
                ppt.slides[leader_metric_slide_index].shapes.add_picture(image_path, x0 + marker_count * delta_x, y0, width=w, height=h)
            marker_count += 1
        except:
            if platform_order[platform_index] == 'Instagram' and image == 'Avg Video Views':
                marker_count += 1
            pass

        if marker_count % 3 == 0:
            if marker_count == 0 and platform_order[platform_index] == 'YouTube':
                pass
            else:
                leader_metric_slide_index += 1
                marker_count = 0

    platform_index += 1
    
    if marker_count != 0:
        leader_metric_slide_index += 2

% Change metrics for vertical bar charts (Campaign)

In [71]:
# Slide 66 - 80
leader_metric_slide_index = 65
platform_index = 0
delta_x = Inches(3.83)
x0 = Inches(3.55) 
y0 = Inches(1.52)
w = Inches(1.99)  
h = Inches(1.24)

while leader_metric_slide_index < 80:
    marker_count = 0
    for index, image in enumerate(ppt_image_order_leader):
        if image == 'Post_distribution' and platform_order[platform_index] != 'YouTube':
                marker_count += 1
        image_path = rf"F:\Projects\PPTX_env\WB_weekly\bar_chart_images\{platform_order[platform_index]}_delta_metrics\Campaign\{image}.png"
        try:
            if image == 'Avg Video Views' and platform_order[platform_index] != 'YouTube':
                ppt.slides[leader_metric_slide_index].shapes.add_picture(image_path, x0 + (marker_count - 0.5) * delta_x, y0, width=w, height=h)
            else:    
                ppt.slides[leader_metric_slide_index].shapes.add_picture(image_path, x0 + marker_count * delta_x, y0, width=w, height=h)
            marker_count += 1
        except:
            if platform_order[platform_index] == 'Instagram' and image == 'Avg Video Views':
                marker_count += 1
            pass

        if marker_count % 3 == 0:
            if marker_count == 0 and platform_order[platform_index] == 'YouTube':
                pass
            else:
                leader_metric_slide_index += 1
                marker_count = 0

    platform_index += 1
    
    if marker_count != 0:
        leader_metric_slide_index += 2

In [72]:
date_list = [(start_date + timedelta(days=i)).strftime('%d-%b') for i in range(7)]

In [88]:
leader_colors = {
    "AITC": "#20C646",       
    "Green": "#00FF00",     
    "BJP": '#f97d09',       
    "Blue": "#0000FF",    
    "INC": '#87CEEB',    
    "Teal": "#008080",      
    "CPIM": "#FF0000"    
}

In [89]:
color_mapping = dict(zip(target_leader_list + ['AITC', 'BJP', 'INC', 'CPI(M)'], list(leader_colors.values()) + ['#20C646', '#f97d09', '#87CEEB', '#FF0000']))
color_mapping

{'Mamata Banerjee': '#20C646',
 'Abhishek Banerjee': '#00FF00',
 'Suvendu Adhikari': '#f97d09',
 'Dilip Ghosh': '#0000FF',
 'Adhir Ranjan Chowdhury': '#87CEEB',
 'Md Salim': '#008080',
 'Sujan Chakraborty': '#FF0000',
 'AITC': '#20C646',
 'BJP': '#f97d09',
 'INC': '#87CEEB',
 'CPI(M)': '#FF0000'}

In [90]:
x1 = Inches(0.15)
x2 = Inches(6.91)
y1 = Inches(1.48)
y2 = Inches(4.50)
w = Inches(6.22)
h =  Inches(2.45)

In [91]:
metric_coord_mapping = {'engagement_rate':(x2, y2), 'moving_post_link':(x1, y1), 'followers':(x2, y1), 'average_engagement':(x1, y2)}

Trendlines Party

In [92]:
starter_slide = 10
for platform_index, platform in enumerate(pd.ExcelFile(r"C:\Users\useer\Downloads\WB_Sep_29_Oct_5_data_dump\WB_report_7_DMA.xlsx").sheet_names):
    dma_df = pd.read_excel(r"C:\Users\useer\Downloads\WB_Sep_29_Oct_5_data_dump\WB_report_7_DMA.xlsx", sheet_name = platform)
    dma_df = pd.merge(dma_df, mapping_df, how='left', left_on = 'presence_handle', right_on = platform)
    foll_df = pd.read_excel(last_7_days_path, sheet_name = platform + '_all_stats_post')

    for handle in list(foll_df['presence_handle'].unique()):
        dma_df.loc[dma_df['presence_handle'] == handle, 'followers'] = foll_df[foll_df['presence_handle'] == handle]['Followers'].values[0]

    dma_df = dma_df[['Name', 'mapping', 'engagement_rate', 'followers', 'moving_post_link','average_engagement']]
    dma_df = dma_df.fillna(0)
    dma_df_party = dma_df[dma_df['mapping'] == 'Party']
    for metric_index, metric in enumerate(list(dma_df.columns[2:])):
        chart_data = CategoryChartData()
        chart_data.categories = [(start_date + timedelta(days=i)).strftime('%d-%b') for i in range(7)]

        platform_party_presence_list = []

        for party in dma_df_party['Name'].unique().tolist():
            if party in list(dma_df_party['Name'].unique()):
                chart_data.add_series(metric, dma_df_party[dma_df_party['Name'] == party].iloc[-7:][metric].tolist())
                platform_party_presence_list.append(party)

        chart = ppt.slides[starter_slide + platform_index].shapes.add_chart(
            XL_CHART_TYPE.LINE, metric_coord_mapping[metric][0], metric_coord_mapping[metric][1], w, h, chart_data
        ).chart
        
        chart.has_legend = False
        chart.value_axis.visible = False
        chart.value_axis.major_gridlines.format.line.fill.background()
        x_axis = chart.category_axis
        x_axis.visible = True  
        x_axis.tick_marks = None
        x_axis.tick_labels.font.size = Pt(10) 
        x_axis.tick_labels.font.name = 'Bahnschrift' 
        x_axis.tick_labels.font.bold = False
        for series_index, series in enumerate(chart.series):
            line = series.format.line
            line.width = Pt(1.5)
            line.color.rgb = hex_to_rgb(color_mapping[platform_party_presence_list[series_index]])
            series.marker.style = XL_MARKER_STYLE.CIRCLE
            series.marker.format.line.color.rgb = hex_to_rgb(color_mapping[platform_party_presence_list[series_index]])
            series.marker.format.line.width = Pt(1.5)
            series.marker.format.fill.solid()
            series.marker.format.fill.fore_color.rgb = hex_to_rgb(color_mapping[platform_party_presence_list[series_index]])

            for point_index, point in enumerate(series.points):
                if metric == 'followers':
                    point.data_label.text_frame.text = mil_k_mapper(dma_df_party[dma_df_party['Name'] == platform_party_presence_list[series_index]].iloc[-7:][metric].tolist()[point_index])

                else:
                    point.data_label.text_frame.text = "{:,.1f}".format(dma_df_party[dma_df_party['Name'] == platform_party_presence_list[series_index]].iloc[-7:][metric].tolist()[point_index])

                text_formatter(point.data_label.text_frame, 8, 'Bahnschrift', PP_ALIGN.CENTER, "#000000")
                point.data_label.position = XL_LABEL_POSITION.ABOVE

Trendlines Leader

In [93]:
starter_slide = 36
for platform_index, platform in enumerate(pd.ExcelFile(r"C:\Users\useer\Downloads\WB_Sep_29_Oct_5_data_dump\WB_report_7_DMA.xlsx").sheet_names):
    dma_df = pd.read_excel(r"C:\Users\useer\Downloads\WB_Sep_29_Oct_5_data_dump\WB_report_7_DMA.xlsx", sheet_name = platform)
    dma_df = pd.merge(dma_df, mapping_df, how='left', left_on = 'presence_handle', right_on = platform)
    foll_df = pd.read_excel(last_7_days_path, sheet_name = platform + '_all_stats_post')

    for handle in list(foll_df['presence_handle'].unique()):
        dma_df.loc[dma_df['presence_handle'] == handle, 'followers'] = foll_df[foll_df['presence_handle'] == handle]['Followers'].values[0]

    dma_df = dma_df[['Name', 'mapping', 'engagement_rate', 'followers', 'moving_post_link','average_engagement']]
    dma_df = dma_df.fillna(0)
    dma_df_leader = dma_df[dma_df['mapping'] == 'Leader']

    for metric_index, metric in enumerate(list(dma_df.columns[2:])):
        chart_data = CategoryChartData()
        chart_data.categories = [(start_date + timedelta(days=i)).strftime('%d-%b') for i in range(7)]

        platform_leader_presence_list = []

        for leader in target_leader_list:
            if leader in list(dma_df_leader['Name'].unique()):
                chart_data.add_series(metric, dma_df_leader[dma_df_leader['Name'] == leader].iloc[-7:][metric].tolist())
                platform_leader_presence_list.append(leader)

        chart = ppt.slides[starter_slide + platform_index].shapes.add_chart(
            XL_CHART_TYPE.LINE, metric_coord_mapping[metric][0], metric_coord_mapping[metric][1], w, h, chart_data
        ).chart
        
        chart.has_legend = False
        chart.value_axis.visible = False
        chart.value_axis.major_gridlines.format.line.fill.background()
        x_axis = chart.category_axis
        x_axis.visible = True  
        x_axis.tick_marks = None
        x_axis.tick_labels.font.size = Pt(10) 
        x_axis.tick_labels.font.name = 'Bahnschrift' 
        x_axis.tick_labels.font.bold = False
        
        for series_index, series in enumerate(chart.series):
            line = series.format.line
            line.width = Pt(1.5)
            line.color.rgb = hex_to_rgb(color_mapping[platform_leader_presence_list[series_index]])
            series.marker.style = XL_MARKER_STYLE.CIRCLE
            series.marker.format.line.color.rgb = hex_to_rgb(color_mapping[platform_leader_presence_list[series_index]])
            series.marker.format.line.width = Pt(1.5)
            series.marker.format.fill.solid()
            series.marker.format.fill.fore_color.rgb = hex_to_rgb(color_mapping[platform_leader_presence_list[series_index]])

            for point_index, point in enumerate(series.points):
                if metric == 'followers':
                    point.data_label.text_frame.text = mil_k_mapper(dma_df_leader[dma_df_leader['Name'] == platform_leader_presence_list[series_index]].iloc[-7:][metric].tolist()[point_index])

                else:
                    point.data_label.text_frame.text = "{:,.1f}".format(dma_df_leader[dma_df_leader['Name'] == platform_leader_presence_list[series_index]].iloc[-7:][metric].tolist()[point_index])

                text_formatter(point.data_label.text_frame, 8, 'Bahnschrift', PP_ALIGN.CENTER, "#000000")
                point.data_label.position = XL_LABEL_POSITION.ABOVE

In [94]:
date_slides = [10, 11, 12, 13, 14, 17, 18, 20, 21, 23, 24, 26, 27, 35, 36, 37, 38, 39, 40, 48, 49, 50, 52, 53, 54, 55, 57, 58, 59, 61, 62, 63, 66, 67, 68, 70, 71, 72, 73, 75, 77, 79, 81]
text = f'*Duration: {start_date.strftime("%d %b, %Y")} – {end_date.strftime("%d %b, %Y")}'

In [95]:
for slide_no in date_slides:
    add_date_textbox(ppt, slide_no - 1, Inches(10.78), Inches(6.85), Inches(1.54), Inches(0.27), text, 10, '#7F6000', 'Bahnschrift')

In [96]:
ppt.save(r"F:\Projects\PPTX_env\WB_weekly\WB_Weekly SMA_test.pptx")